# Q&A with Rag using Local Model

### Document Loading

First, install packages needed for local embeddings and vector storage.

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma pypdf langchain-google-vertexai google-cloud-storage

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Indexing: Load

Use PyPDFLoader as in this [document](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/).

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/sample_data/Owner-Manual-KICKS-e-POWER-EN.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

### Indexing: Split

To handle this we’ll split the Document into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the manual at run time.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

Next, the below steps will download the GPT4All embeddings locally (if you don't already have them).

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

### Retrieval and Generation: Retrieve

Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What is auto brake hold?")
len(retrieved_docs)

In [ ]:
print(retrieved_docs[0].page_content)

### Retrieval and Generation: Generate
Let’s put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output.

Install dependencies

In [ ]:
# Authenticate
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Set environment variables

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_CLOUD_PROJECT"] = getpass.getpass()

In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-pro")

Let’s put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is auto brake hold and how does function be?")